In [1]:
from binance.um_futures import UMFutures
import plotly.graph_objects as go
import requests
import json
import numpy as np
import pandas as pd
from pandas import Timestamp
from sklearn.linear_model import LinearRegression
import datetime
from chart import Chart
um_futures_client = UMFutures()
um_futures_client = UMFutures("iPQMe46exUy10KiBaBahQT7ow1uzb9jaxlKj19Bg5BI8JEwJL5bw9LCvJtfKuVbP","J4FGJQdGPpNGm8jpnj3IuFuDJuAe7FIa3cM51L4z662UYux3qD0ByLJ0bfIltZvK")

Source = https://github.com/binance/binance-futures-connector-python/tree/main (UMFutures)

Documentation = https://binance-docs.github.io/apidocs/futures/en/#public-endpoints-info (UMFutures)

In [2]:
def convert_to_df(data):
    data_df=pd.DataFrame(np.asarray(data,dtype=np.float64),columns=["Open Time","Open","High","Low","Close","Volume","Close Time","Quote Asset Volume","Number of Trades","Taker Buy Volume","Taker Buy Quote Asset Volume","Ignore"])
    data_df['Open Time'] = pd.to_datetime(data_df['Open Time'], unit='ms')
    data_df['Close Time'] = pd.to_datetime(data_df["Close Time"],unit='ms').dt.ceil("min")
    return data_df.drop("Ignore",axis=1)

In [3]:
data=convert_to_df(um_futures_client.continuous_klines("CHZUSDT",contractType="PERPETUAL", interval="15m"))
chz_chart = Chart(data)

In [4]:
chz_chart.plot()

In [5]:
line = chz_chart.gradient_descent(data)

In [6]:
chz_chart.check_oc_collisions(0,450,line)

[[Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00'), False],
 [Timestamp('2024-02-11 12:00:00